In [1]:
import logging
from sys import stdout

formatter = logging.Formatter(
    '%(asctime)s - %(name)s - %(levelname)s - %(message)s')
console_handler = logging.StreamHandler(stdout)
console_handler.setFormatter(formatter)

logger = logging.getLogger('opensky.producer')
logger.addHandler(console_handler)
logger.setLevel('DEBUG')



In [128]:
from opensky_api import OpenSkyApi
from kafka import KafkaProducer
import json
from time import sleep
import pyarrow as pa
import os
from datetime import datetime
import shutil
    
# # Topics/Brokers
topic_real_time_states = 'real-time-states'
topic_raw_json = 'raw_json'
topic_flat_json = 'flat_json'
topic_brokers = ['localhost:9092']


size_mb = lambda x : x/1024/1024

API = OpenSkyApi('livgeni', '1abc23')

def open_sky_generator():
    while True:
        yield API.get_states()
        sleep(10)


RED_FIELDS = ['origin_country', 'sensors']
YELLOW_FIELDS = ['time_position', 'heading', 'vertical_rate', 'spi']
IGNORE_FIELDS = RED_FIELDS + YELLOW_FIELDS


def opensky_to_dict(opensky_states):
    """returns a list of dictionaries"""
    svdl = list()

    for sv in opensky_states.states:
        svd = dict(time = opensky_states.time)
        for key in sv.keys:
            if key not in IGNORE_FIELDS:
                val = sv.__dict__[key]
                svd[key] = val if type(val) != str else val.strip()
        svdl += [svd]
    return svdl
        
#     state_vector_json_list = json.dumps(svdl)
#     return state_vector_json_list


        
class OpenskyArchiver:
    
    def __init__(self):
        # hdfs config
        self.hdfs_config = dict(
                            hdfs_host='localhost',
                            hdfs_port=8020,
                            hdfs_user='hdfs',
                            hdfs_driver='libhdfs',
                            hdfs_archive_path = '/FinalProject/Archive'
                               )
        # local file configs
        self.local_files_folder = os.path.abspath('/home/naya/tutorial/open-sky/tmp_storage')
        self.local_files_prefix = 'opensky_state_vectors_json'
        self.local_file_size_threshold = 100 #MB
        self.gen_time_suffix = lambda : datetime.now().strftime("%Y-%m-%d_%H_%M_%S")
        self.gen_local_file_path = lambda : os.path.join\
                                    (self.local_files_folder,\
                                     f'{self.local_files_prefix}_{self.gen_time_suffix()}.json')
        self.local_file_path = self.gen_local_file_path()
        
        # init local and hdfs folders
        fs = pa.hdfs.connect(
            host=self.hdfs_config['hdfs_host'],
            port=self.hdfs_config['hdfs_port'], 
            user=self.hdfs_config['hdfs_user'], 
            kerb_ticket=None, 
            driver=self.hdfs_config['hdfs_driver'], 
            extra_conf=None)

        # Create local folder if not exists
        if os.path.exists(self.local_files_folder):
            shutil.rmtree(self.local_files_folder)
        os.makedirs(self.local_files_folder)

        # create hdfs folder if not exist
        hdfs_archive_path = self.hdfs_config['hdfs_archive_path']
        if not fs.exists(hdfs_archive_path):
            fs.mkdir(hdfs_archive_path, create_parents=True)
        
        self.fs = fs
        
    def archive_data(self, jsn_str):
        """
        load as json current file if exists, and append the new json string and dump back to the file
        if file does not exist create new and dump the new file    
        """ 
        if os.path.isfile(self.local_file_path) and os.path.getsize(self.local_file_path) > 0:
            with open(self.local_file_path, 'r') as local_file:
                file_jsn = json.load(local_file)
        else:
            file_jsn = json.loads('[]')
        with open(self.local_file_path, 'w') as local_file:
            new_jsn = json.loads(jsn_str)
            file_jsn.extend(new_jsn)
            json.dump(file_jsn, local_file)
#             local_file.write(jsn_str)
        # test if target file size reached
        if size_mb(os.path.getsize(self.local_file_path)) >= self.local_file_size_threshold:
            # upload to hdfs
            logger.debug(f'Uploading to HDFS')
            with open(self.local_file_path, 'rb') as source:
                target_file_name = os.path.split(self.local_file_path)[1]
                target_full_path = f'{self.hdfs_config["hdfs_archive_path"]}/{target_file_name}'
                self.fs.upload(target_full_path, source)
            logger.debug(f'Uploaded to : {target_full_path}')
            # delete the local file
            logger.debug(f'deleting local file {self.local_file_path}')
            os.remove(self.local_file_path)
            # new name for new file
            self.local_file_path = self.gen_local_file_path()
            logger.debug(f'generated new local_file_name :{os.path.split(self.local_file_path)[1]}')

## Save to Archiving Folder in HDFS

1. Append to a local file until reaching 100MB
2. When reached - upload to HDFS using pyarrow and open a new file

In [ ]:
producer = KafkaProducer(bootstrap_servers=topic_brokers)

archiver = OpenskyArchiver()

for states in open_sky_generator():
    message_json = json.dumps(opensky_to_dict(states))
    archiver.archive_data(message_json)
#    producer.send(topic_raw_json, value = message_json[:900000].encode('utf-8'))
#     producer.send(topic_raw_json, b'message')
    logger.debug(f"len : {len(message_json)}")

2019-12-13 21:59:46,616 - opensky.producer - DEBUG - len : 2098193
2019-12-13 22:00:00,764 - opensky.producer - DEBUG - len : 2096398
2019-12-13 22:00:15,223 - opensky.producer - DEBUG - len : 2100638
2019-12-13 22:00:30,525 - opensky.producer - DEBUG - len : 2100898
2019-12-13 22:00:44,760 - opensky.producer - DEBUG - len : 2101922
2019-12-13 22:00:58,856 - opensky.producer - DEBUG - len : 2104466
2019-12-13 22:01:13,095 - opensky.producer - DEBUG - len : 2103781
2019-12-13 22:01:28,472 - opensky.producer - DEBUG - len : 2104294
2019-12-13 22:01:43,386 - opensky.producer - DEBUG - len : 2101738
2019-12-13 22:01:58,567 - opensky.producer - DEBUG - len : 2106748
2019-12-13 22:02:14,324 - opensky.producer - DEBUG - len : 2110789
2019-12-13 22:02:30,264 - opensky.producer - DEBUG - len : 2102591
2019-12-13 22:02:47,287 - opensky.producer - DEBUG - len : 2102674
2019-12-13 22:03:04,187 - opensky.producer - DEBUG - len : 2102889
2019-12-13 22:03:22,213 - opensky.producer - DEBUG - len : 210

2019-12-13 22:33:21,319 - opensky.producer - DEBUG - len : 2130659
2019-12-13 22:33:36,285 - opensky.producer - DEBUG - len : 2127934
2019-12-13 22:33:51,864 - opensky.producer - DEBUG - len : 2126360
2019-12-13 22:34:08,313 - opensky.producer - DEBUG - len : 2130563
2019-12-13 22:34:24,819 - opensky.producer - DEBUG - len : 2126800
2019-12-13 22:34:40,950 - opensky.producer - DEBUG - len : 2133537
2019-12-13 22:34:56,479 - opensky.producer - DEBUG - len : 2136599
2019-12-13 22:35:12,471 - opensky.producer - DEBUG - len : 2138230
2019-12-13 22:35:29,303 - opensky.producer - DEBUG - len : 2138129
2019-12-13 22:35:46,820 - opensky.producer - DEBUG - len : 2138909
2019-12-13 22:36:04,682 - opensky.producer - DEBUG - len : 2134398
2019-12-13 22:36:22,547 - opensky.producer - DEBUG - len : 2128807
2019-12-13 22:36:41,655 - opensky.producer - DEBUG - len : 2128188
2019-12-13 22:36:58,895 - opensky.producer - DEBUG - len : 2130297
2019-12-13 22:37:17,226 - opensky.producer - DEBUG - len : 213

2019-12-13 23:06:57,267 - opensky.producer - DEBUG - len : 2099137
2019-12-13 23:07:14,041 - opensky.producer - DEBUG - len : 2103531
2019-12-13 23:07:30,291 - opensky.producer - DEBUG - len : 2103575
2019-12-13 23:07:48,512 - opensky.producer - DEBUG - len : 2098905
2019-12-13 23:08:05,162 - opensky.producer - DEBUG - len : 2091255
2019-12-13 23:08:22,718 - opensky.producer - DEBUG - len : 2093599
2019-12-13 23:08:40,132 - opensky.producer - DEBUG - len : 2090660
2019-12-13 23:08:57,882 - opensky.producer - DEBUG - len : 2088728
2019-12-13 23:09:17,398 - opensky.producer - DEBUG - len : 2082129
2019-12-13 23:09:36,576 - opensky.producer - DEBUG - len : 2077631
2019-12-13 23:09:55,307 - opensky.producer - DEBUG - len : 2078665
2019-12-13 23:10:14,009 - opensky.producer - DEBUG - len : 2076044
2019-12-13 23:10:33,212 - opensky.producer - DEBUG - len : 2076882
2019-12-13 23:10:52,419 - opensky.producer - DEBUG - len : 2075915
2019-12-13 23:11:12,809 - opensky.producer - DEBUG - len : 208

2019-12-13 23:41:50,095 - opensky.producer - DEBUG - len : 2015579
2019-12-13 23:42:07,771 - opensky.producer - DEBUG - len : 2013225
2019-12-13 23:42:26,911 - opensky.producer - DEBUG - len : 2007750
2019-12-13 23:42:47,021 - opensky.producer - DEBUG - len : 2004469
2019-12-13 23:43:05,829 - opensky.producer - DEBUG - len : 2001474
2019-12-13 23:43:25,210 - opensky.producer - DEBUG - len : 2000384
2019-12-13 23:43:45,471 - opensky.producer - DEBUG - len : 1994642
2019-12-13 23:44:05,536 - opensky.producer - DEBUG - len : 1989914
2019-12-13 23:44:25,390 - opensky.producer - DEBUG - len : 1988360
2019-12-13 23:44:44,992 - opensky.producer - DEBUG - len : 1983728
2019-12-13 23:45:06,606 - opensky.producer - DEBUG - len : 1988975
2019-12-13 23:45:27,637 - opensky.producer - DEBUG - len : 1988919
2019-12-13 23:45:47,640 - opensky.producer - DEBUG - len : 1983917
2019-12-13 23:46:09,036 - opensky.producer - DEBUG - len : 1991324
2019-12-13 23:46:29,569 - opensky.producer - DEBUG - len : 198

2019-12-14 00:15:21,855 - opensky.producer - DEBUG - len : 1923323
2019-12-14 00:15:39,677 - opensky.producer - DEBUG - len : 1922014
2019-12-14 00:15:59,123 - opensky.producer - DEBUG - len : 1916836
2019-12-14 00:16:18,838 - opensky.producer - DEBUG - len : 1916397
2019-12-14 00:16:37,010 - opensky.producer - DEBUG - len : 1920000
2019-12-14 00:16:55,464 - opensky.producer - DEBUG - len : 1917475
2019-12-14 00:17:15,633 - opensky.producer - DEBUG - len : 1915447
2019-12-14 00:17:35,740 - opensky.producer - DEBUG - len : 1909353
2019-12-14 00:17:55,058 - opensky.producer - DEBUG - len : 1908429
2019-12-14 00:18:14,230 - opensky.producer - DEBUG - len : 1905621
2019-12-14 00:18:33,796 - opensky.producer - DEBUG - len : 1906911
2019-12-14 00:18:55,114 - opensky.producer - DEBUG - len : 1899918
2019-12-14 00:19:16,206 - opensky.producer - DEBUG - len : 1900647
2019-12-14 00:19:36,981 - opensky.producer - DEBUG - len : 1899419
2019-12-14 00:19:57,647 - opensky.producer - DEBUG - len : 190

...

2019-12-14 09:30:31,194 - opensky.producer - DEBUG - len : 1140979
2019-12-14 09:30:50,294 - opensky.producer - DEBUG - len : 1140493
2019-12-14 09:31:09,536 - opensky.producer - DEBUG - len : 1143258
2019-12-14 09:31:28,965 - opensky.producer - DEBUG - len : 1142900
2019-12-14 09:31:48,566 - opensky.producer - DEBUG - len : 1142549
2019-12-14 09:32:08,293 - opensky.producer - DEBUG - len : 1144172
2019-12-14 09:32:29,160 - opensky.producer - DEBUG - len : 1146290
2019-12-14 09:32:48,944 - opensky.producer - DEBUG - len : 1147293
2019-12-14 09:33:09,246 - opensky.producer - DEBUG - len : 1147933
2019-12-14 09:33:29,469 - opensky.producer - DEBUG - len : 1146331
2019-12-14 09:33:50,250 - opensky.producer - DEBUG - len : 1149318
2019-12-14 09:34:10,776 - opensky.producer - DEBUG - len : 1145317
2019-12-14 09:34:31,310 - opensky.producer - DEBUG - len : 1145432
2019-12-14 09:34:51,715 - opensky.producer - DEBUG - len : 1147640
2019-12-14 09:35:12,441 - opensky.producer - DEBUG - len : 114

2019-12-14 10:05:10,670 - opensky.producer - DEBUG - len : 1161633
2019-12-14 10:05:32,861 - opensky.producer - DEBUG - len : 1155679
2019-12-14 10:05:54,646 - opensky.producer - DEBUG - len : 1152787
2019-12-14 10:06:16,227 - opensky.producer - DEBUG - len : 1153602
2019-12-14 10:06:38,383 - opensky.producer - DEBUG - len : 1158507
2019-12-14 10:07:01,260 - opensky.producer - DEBUG - len : 1157189
2019-12-14 10:07:23,291 - opensky.producer - DEBUG - len : 1156200
2019-12-14 10:07:46,180 - opensky.producer - DEBUG - len : 1150430
2019-12-14 10:08:08,340 - opensky.producer - DEBUG - Uploading to HDFS
2019-12-14 10:08:08,674 - opensky.producer - DEBUG - Uploaded to : /FinalProject/Archive/opensky_state_vectors_json_2019-12-14_09_41_25.json
2019-12-14 10:08:08,675 - opensky.producer - DEBUG - deleting local file /home/naya/tutorial/open-sky/tmp_storage/opensky_state_vectors_json_2019-12-14_09_41_25.json
2019-12-14 10:08:08,692 - opensky.producer - DEBUG - generated new local_file_name :op

2019-12-14 10:36:20,719 - opensky.producer - DEBUG - len : 1155286
2019-12-14 10:36:33,474 - opensky.producer - DEBUG - len : 1149763
2019-12-14 10:36:46,335 - opensky.producer - DEBUG - len : 1147511
2019-12-14 10:36:59,210 - opensky.producer - DEBUG - len : 1147052
2019-12-14 10:37:13,212 - opensky.producer - DEBUG - len : 1145709
2019-12-14 10:37:26,830 - opensky.producer - DEBUG - len : 1147594
2019-12-14 10:37:40,376 - opensky.producer - DEBUG - len : 1147559
2019-12-14 10:37:53,894 - opensky.producer - DEBUG - len : 1147241
2019-12-14 10:38:07,335 - opensky.producer - DEBUG - len : 1149439
2019-12-14 10:38:21,096 - opensky.producer - DEBUG - len : 1148628
2019-12-14 10:38:34,881 - opensky.producer - DEBUG - len : 1144185
2019-12-14 10:38:49,000 - opensky.producer - DEBUG - len : 1145842
2019-12-14 10:39:03,397 - opensky.producer - DEBUG - len : 1143560
2019-12-14 10:39:18,322 - opensky.producer - DEBUG - len : 1141864
2019-12-14 10:39:33,028 - opensky.producer - DEBUG - len : 114

2019-12-14 11:08:39,707 - opensky.producer - DEBUG - len : 1166610
2019-12-14 11:08:55,634 - opensky.producer - DEBUG - len : 1166818
2019-12-14 11:09:11,766 - opensky.producer - DEBUG - len : 1163065
2019-12-14 11:09:27,622 - opensky.producer - DEBUG - len : 1162840
2019-12-14 11:09:44,533 - opensky.producer - DEBUG - len : 1163880
2019-12-14 11:10:00,416 - opensky.producer - DEBUG - len : 1163701
2019-12-14 11:10:16,442 - opensky.producer - DEBUG - len : 1162565
2019-12-14 11:10:32,595 - opensky.producer - DEBUG - len : 1165137
2019-12-14 11:10:49,120 - opensky.producer - DEBUG - len : 1168070
2019-12-14 11:11:05,690 - opensky.producer - DEBUG - len : 1169062
2019-12-14 11:11:22,538 - opensky.producer - DEBUG - len : 1167870
2019-12-14 11:11:39,644 - opensky.producer - DEBUG - len : 1172607
2019-12-14 11:11:56,373 - opensky.producer - DEBUG - len : 1169417
2019-12-14 11:12:13,833 - opensky.producer - DEBUG - len : 1167463
2019-12-14 11:12:33,058 - opensky.producer - DEBUG - len : 116

2019-12-14 11:42:12,610 - opensky.producer - DEBUG - len : 1203245
2019-12-14 11:42:31,291 - opensky.producer - DEBUG - len : 1206330
2019-12-14 11:42:50,383 - opensky.producer - DEBUG - len : 1206918
2019-12-14 11:43:09,462 - opensky.producer - DEBUG - len : 1206688
2019-12-14 11:43:28,801 - opensky.producer - DEBUG - len : 1205765
2019-12-14 11:43:47,937 - opensky.producer - DEBUG - len : 1202965
2019-12-14 11:44:08,614 - opensky.producer - DEBUG - len : 1204459
2019-12-14 11:44:28,299 - opensky.producer - DEBUG - len : 1204552
2019-12-14 11:44:48,661 - opensky.producer - DEBUG - len : 1207716
2019-12-14 11:45:08,204 - opensky.producer - DEBUG - len : 1208305
2019-12-14 11:45:28,317 - opensky.producer - DEBUG - len : 1204091
2019-12-14 11:45:48,334 - opensky.producer - DEBUG - len : 1201568
2019-12-14 11:46:08,639 - opensky.producer - DEBUG - len : 1201208
2019-12-14 11:46:29,462 - opensky.producer - DEBUG - len : 1206330
2019-12-14 11:46:49,796 - opensky.producer - DEBUG - len : 120

In [87]:
local_file_path = os.path.join(local_files_folder, local_files_prefix+'_'+local_files_suffix())
local_file_path

'/home/naya/tutorial/open-sky/tmp_storage/opensky_state_vectors_json_2019-08-12_21_08_48'

In [46]:
local_file_path = os.path.join\
(local_files_folder, f'{local_files_prefix}_{local_files_suffix()}.json')

In [49]:
import pyarrow as pa
import json

hdfs_config = dict(\
                            hdfs_host='localhost',\
                            hdfs_port=8020,\
                            hdfs_user='hdfs',\
                            hdfs_driver='libhdfs',\
                            hdfs_archive_path = '/FinalProject/Archive'\
                               )
fs = pa.hdfs.connect(
    host=hdfs_config['hdfs_host'],
    port=hdfs_config['hdfs_port'], 
    user=hdfs_config['hdfs_user'], 
    kerb_ticket=None, 
    driver=hdfs_config['hdfs_driver'], 
    extra_conf=None)

In [120]:
folder = '/FinalProject/Archive'

file_path = fs.ls(folder)[-1]

print(file_path)

with fs.open(file_path,'rb') as j:
     jsn = json.load(j)

len(jsn)

/FinalProject/Archive/opensky_state_vectors_json_2019-12-13_21_41_54.json


32134

In [110]:

#     jsn = json.loads(txt)



jsn1 = json.loads('[{"time": 1576262559, "icao24": "e8027d", "callsign": "LPE2104", "last_contact": 1576262429, "longitude": -73.5805, "latitude": -15.6016, "baro_altitude": 10972.8, "on_ground": false, "velocity": 231.14, "geo_altitude": 11719.56, "squawk": "1673", "position_source": 0}, {"time": 1576262559, "icao24": "ad4f1d", "callsign": "N957ES", "last_contact": 1576262559, "longitude": -96.1682, "latitude": 35.9688, "baro_altitude": 1379.22, "on_ground": false, "velocity": 47.7, "geo_altitude": 1318.26, "squawk": "4724", "position_source": 0}, {"time": 1576262559, "icao24": "aa56da", "callsign": "UAL1460", "last_contact": 1576262559, "longitude": -77.0063, "latitude": 38.4849, "baro_altitude": 7620, "on_ground": false, "velocity": 252.07, "geo_altitude": 7749.54, "squawk": "3372", "position_source": 0}]')
jsn2 = json.loads('[{"time": 1576262420, "icao24": "e8027d", "callsign": "LPE2104", "last_contact": 1576262429, "longitude": -73.5805, "latitude": -15.6016, "baro_altitude": 10972.8, "on_ground": false, "velocity": 231.14, "geo_altitude": 11719.56, "squawk": "1673", "position_source": 0}, {"time": 1576262559, "icao24": "ad4f1d", "callsign": "N957ES", "last_contact": 1576262559, "longitude": -96.1682, "latitude": 35.9688, "baro_altitude": 1379.22, "on_ground": false, "velocity": 47.7, "geo_altitude": 1318.26, "squawk": "4724", "position_source": 0}, {"time": 1576262559, "icao24": "aa56da", "callsign": "UAL1460", "last_contact": 1576262559, "longitude": -77.0063, "latitude": 38.4849, "baro_altitude": 7620, "on_ground": false, "velocity": 252.07, "geo_altitude": 7749.54, "squawk": "3372", "position_source": 0}]')
print(len(jsn1))
jsn1.extend(jsn2)
print(len(jsn1))
jsn1.extend(jsn1)
print(len(jsn1))

fp = '/home/naya/tutorial/open-sky/tmp_storage/opensky_state_vectors_json_2019-12-13_21_13_26.json'

out = '/home/naya/tutorial/open-sky/tmp_storage/out.json'

with open(fp,'w+') as f:
    jsn = json.load(f) if os.path.getsize(fp) > 0 else json.loads('[]')
    print(jsn)
    print(len(jsn))
    print(len(jsn1))
    jsn.extend(jsn1)
    print(len(jsn))
#     print(len(jsn))
#     jsn.extend(jsn1)
#     print(len(jsn))
# with open(out,'w') as fp:
#     json.dump(jsn, fp)



3
6
12
[]
0
12
12
